In [2]:
!pip install -qqq crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.3/209.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from crewai import Agent, Crew, Process, Task
from crewai_tools import PDFSearchTool
import os


In [5]:
from google.colab import userdata
openai_api_key = userdata.get('openai-key')
os.environ['OPENAI_API_KEY'] = openai_api_key

In [7]:
pdf_tool_1 = PDFSearchTool(
pdf='/content/drive/MyDrive/Sandeep Resume PDF Version.pdf',
config=dict(
llm=dict(
provider="openai",
),
embedder=dict(
provider="openai",
config=dict(
model="text-embedding-3-large",
),
),
)
)


Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.43it/s]


In [8]:
pdf_tool_2 = PDFSearchTool(
pdf='/content/drive/MyDrive/ENCRYTION.pdf',
config=dict(
llm=dict(
provider="openai",
config=dict(
model="gpt-3.5-turbo",
),
),
embedder=dict(
provider="openai",
config=dict(
model="text-embedding-3-large",
),
),
)
)

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.18it/s]


In [9]:
pdf_tool_3 = PDFSearchTool(
pdf='/content/drive/MyDrive/Sandeep_Kumar_CV.pdf',
config=dict(
llm=dict(
provider="openai",
config=dict(
model="gpt-3.5-turbo",
),
),
embedder=dict(
provider="openai",
config=dict(
model="text-embedding-3-large",
),
),
)
)

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.13it/s]


In [10]:
pdf_agent_1 = Agent(
role="PDF 1 Retrieval Specialist",
goal="Extract relevant information from PDF 1",
allow_delegation=False,
verbose=True,
backstory=(
"""
This agent is responsible for finding and providing information
from PDF 1, ensuring accuracy and precision in responses.
"""
),
tools=[pdf_tool_1],
)


In [11]:
pdf_agent_2 = Agent(
role="PDF 2 Retrieval Specialist",
goal="Extract relevant information from PDF 2",
allow_delegation=False,
verbose=True,
backstory=(
"""
This agent is responsible for finding and providing information
from PDF 2, ensuring accuracy and precision in responses.
"""
),
tools=[pdf_tool_2],
)


In [12]:
pdf_agent_2 = Agent(
role="PDF 2 Retrieval Specialist",
goal="Extract relevant information from PDF 2",
allow_delegation=False,
verbose=True,
backstory=(
"""
This agent is responsible for finding and providing information
from PDF 2, ensuring accuracy and precision in responses.
"""
),
tools=[pdf_tool_2],
)


In [19]:
query_manager = Agent(
role="Question Routing Manager",
goal="Direct questions to the appropriate PDF retrieval agent",
allow_delegation=True,
verbose=True,
backstory=(
"""
This agent identifies which PDF is best suited to answer a question
and routes the task to the relevant retrieval agent.
"""
),
tools=[],
)

In [20]:
task_pdf_1 = Task(
description=(
"""
Utilize PDF 1 to provide answers to the question.
Ensure that the answers are clear, accurate, and based on PDF 1.
Question:
{question}
"""
),
expected_output="""
Accurate answers derived from the content of PDF 1.
""",
tools=[pdf_tool_1],
agent=pdf_agent_1,
)

In [21]:
task_pdf_2 = Task(
description=(
"""
Utilize PDF 2 to provide answers to the question.
Ensure that the answers are clear, accurate, and based on PDF 2.
Question:
{question}
"""
),
expected_output="""
Accurate answers derived from the content of PDF 2.
""",
tools=[pdf_tool_2],
agent=pdf_agent_2,
)


In [22]:
task_pdf_3 = Task(
description=(
"""
Utilize PDF 3 to provide answers to the question.
Ensure that the answers are clear, accurate, and based on PDF 3.
Question:
{question}
"""
),
expected_output="""
Accurate answers derived from the content of PDF 3.
""",
tools=[pdf_tool_3],
agent=pdf_agent_3,
)


In [23]:
task_route_question = Task(
description=(
"""
Identify the most appropriate PDF for the question and
assign the task to the correct retrieval agent.
Question:
{question}
"""
),
expected_output="""
Determine which PDF is relevant and assign the retrieval task accordingly.
""",
tools=[],
agent=query_manager,
)


In [32]:
team = Crew(
    tasks = [task_route_question, task_pdf_1, task_pdf_2, task_pdf_3],
    agents = [query_manager, pdf_agent_1, pdf_agent_2, pdf_agent_3],
    process = Process.sequential,
)
From where did sandeep complete his undergraduation

In [35]:
user_question = input("Please ask your question?\n")
result = team.kickoff(inputs={"question": user_question})
print(result)


Please ask your question?
From where did sandeep complete his undergraduation?


> Entering new CrewAgentExecutor chain...
To determine which PDF is most relevant for answering the question "From where did Sandeep complete his undergraduation?", I need to gather information from the PDF Retrieval Specialists. This will allow me to identify which PDF contains the necessary information.

I will ask each PDF Retrieval Specialist if their PDF contains information about Sandeep's undergraduate education. 

Action: Ask question to coworker
Action Input: {"question": "Does your PDF contain information about where Sandeep completed his undergraduation?", "context": "We need to identify the PDF that contains details about Sandeep's undergraduate education.", "coworker": "PDF 1 Retrieval Specialist"}


> Entering new CrewAgentExecutor chain...
I need to determine if the PDF contains information about where Sandeep completed his undergraduation. To do this, I will search for keywords related to h